In [1]:
import pyodbc
import mysql.connector
import dask.dataframe as dd
import pandas as pd
# from scripts.db_utils import create_ebay_connection
import os 
from dotenv import load_dotenv

load_dotenv()

def create_ebay_connection():
    """Create connection to the hosted SQL Server database for IP addresses."""
    try:
        connection = pyodbc.connect(
            f"DRIVER={{MySQL ODBC 9.1 Unicode Driver}};"
            f"SERVER={os.getenv('EBAY_DB_HOST')};"
            f"DATABASE={os.getenv('EBAY_DB_NAME')};"
            f"UID={os.getenv('EBAY_DB_USER')};"
            f"PWD={os.getenv('EBAY_DB_PASSWORD')}"
        )
        return connection
    except pyodbc.Error as e:
        print(f"Error connecting to hosted database: {e}")
        return None
# Database connection parameters
conn = create_ebay_connection()

query = "SELECT * FROM tbl_iptrace_user_activity where ams_trans_rsn_cd=0"




def fetch_data_in_chunks(conn, query, chunk_size=100000):
    """
    Fetch data from the database in chunks, then convert to Dask DataFrame.
    """
    offset = 0
    while True:
        # Modify the query to include pagination using LIMIT and OFFSET for chunks
        paginated_query = f"{query} LIMIT {chunk_size} OFFSET {offset}"
        
        # Fetch data in chunks using pandas (it handles column and row alignment internally)
        df_chunk = pd.read_sql_query(paginated_query, conn)
        
        if df_chunk.empty:
            break
        
        # Convert to Dask DataFrame
        dask_df = dd.from_pandas(df_chunk, npartitions=4)
        
        yield dask_df
        
        offset += chunk_size


dask_chunks = []

# Fetch chunks and store them in the list
for dask_df_chunk in fetch_data_in_chunks(conn, query, chunk_size=100000):
    dask_chunks.append(dask_df_chunk)

# Concatenate all chunks into a single Dask DataFrame
full_dask_df = dd.concat(dask_chunks)

# Now you can perform operations on full_dask_df
# result = full_dask_df.groupby('some_column').agg({'another_column': 'mean'})

# Compute the result
# computed_result = result.compute()

# Show the result
print(full_dask_df.head())


C:\Users\ramegupta\AppData\Local\Temp\ipykernel_15936\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\AppData\Local\Temp\ipykernel_15936\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\AppData\Local\Temp\ipykernel_15936\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\

   id     CLNT_RMT_IP  ROI_CLICK_EVENT_TS  \
0   1  77.111.247.168 2020-08-02 18:37:01   
1   2   66.249.73.101 2020-08-21 03:36:38   
2   3  77.111.247.129 2020-08-21 03:36:38   
3   4   66.249.73.101 2020-08-21 03:36:33   
4   5  77.111.247.129 2020-08-21 03:36:33   

                                          BRWSR_NAME  ams_trans_rsn_cd  \
0  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
1  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
2  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
3  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
4  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   

  ams_pblshr_id    buyer_id  is_vpn            click_id  
0    5574672411   valen_949       1  209245484389035777  
1    5574672411  morbanisaf       1  209453736892211586  
2    5574672411  morbanisaf       1  209453736892211586  
3    5574672411  morbanisaf       1  209453736892211

In [3]:
# Get the shape of the DataFrame (number of rows and columns)
num_rows, num_columns = full_dask_df.shape[0].compute(), full_dask_df.shape[1]

print(f"Rows: {num_rows}, Columns: {num_columns}")


Rows: 1254243, Columns: 9
